In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.master('local').appName('Uber_DB').config("spark.driver.memory", "15g").config("spark.mongodb.read.connection.uri", "mongodb://localhost:27017/ETL").config("spark.mongodb.write.connection.uri", "mongodb://localhost:27017/ETL").config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector:10.0.2').getOrCreate()

In [3]:
df = spark.read.format('mongodb').option('uri','mongodb://localhost:27017/ETL').option('database','ETL').option('collection','UBER').option('header',True).load()

In [4]:
def findDate(val):
    dt = val.split(" ")
    correct_date = dt[0].replace('-','/',2)
    return correct_date
def findTime(val):
    dt = val.split(" ")
    correct_time = dt[1].split(":")
    return correct_time[0]

In [5]:
today = '11/7/2016' #date.today()

In [6]:
from pyspark.sql.functions import *
df.show()

+---------+------------+----------+-------------------+--------------+--------------------+
|Driver id|Pickup point|Request id|  Request timestamp|        Status|                 _id|
+---------+------------+----------+-------------------+--------------+--------------------+
|        1|     Airport|       619|    11/7/2016 11:51|Trip Completed|64987d005947eb5c0...|
|        1|     Airport|       867|    11/7/2016 17:57|Trip Completed|64987d005947eb5c0...|
|        1|        City|      1807|     12/7/2016 9:17|Trip Completed|64987d005947eb5c0...|
|        1|     Airport|      2532|    12/7/2016 21:08|Trip Completed|64987d005947eb5c0...|
|        1|        City|      3112|13-07-2016 08:33:16|Trip Completed|64987d005947eb5c0...|
|        1|     Airport|      3879|13-07-2016 21:57:28|Trip Completed|64987d005947eb5c0...|
|        1|     Airport|      4270|14-07-2016 06:15:32|Trip Completed|64987d005947eb5c0...|
|        1|     Airport|      5510|15-07-2016 05:11:52|Trip Completed|64987d0059

In [7]:
df.printSchema()
from pyspark.sql.types import StringType
transform_date = udf(findDate)
transform_time = udf(findTime)


root
 |-- Driver id: string (nullable = true)
 |-- Pickup point: string (nullable = true)
 |-- Request id: integer (nullable = true)
 |-- Request timestamp: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- _id: string (nullable = true)



In [8]:
final_df = df.filter(transform_date(col('Request timestamp'))==today).groupBy([col('Status'),transform_date(col('Request timestamp'))]).agg(count(col("Status")))

In [9]:
final_df.show()

+-----------------+---------------------------+-------------+
|           Status|findDate(Request timestamp)|count(Status)|
+-----------------+---------------------------+-------------+
|        Cancelled|                  11/7/2016|          262|
|No Cars Available|                  11/7/2016|          504|
|   Trip Completed|                  11/7/2016|          601|
+-----------------+---------------------------+-------------+



In [12]:
final_df.write.format("mongodb").option("uri","mongodb://localhost:27017/ETL").option("database","ETL").option("collection","Uber_FIlter_Data").mode("append").save()